In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Define the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output[:, -1, :])
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))

In [ ]:
# Preprocess the input text
def preprocess(text):
    chars = list(set(text))
    char_to_index = {char: i for i, char in enumerate(chars)}
    index_to_char = {i: char for i, char in enumerate(chars)}
    return chars, char_to_index, index_to_char

In [ ]:
# Generate training data
def generate_training_data(text, seq_length):
    input_seqs = []
    target_seqs = []
    for i in range(len(text) - seq_length):
        input_seq = text[i:i+seq_length]
        target_seq = text[i+seq_length]
        input_seqs.append([char_to_index[char] for char in input_seq])
        target_seqs.append(char_to_index[target_seq])
    return input_seqs, target_seqs

In [ ]:
# Hyperparameters
seq_length = 100
hidden_size = 256
num_layers = 2
learning_rate = 0.001
num_epochs = 50
batch_size = 64

In [ ]:
# Preprocess the input text
filename = "Deep Learning.txt"
text = open(filename, 'r', encoding='utf-8').read()
chars, char_to_index, index_to_char = preprocess(text)
input_seqs, target_seqs = generate_training_data(text, seq_length)

In [ ]:
# Convert data to PyTorch tensors
input_seqs = torch.tensor(input_seqs, dtype=torch.long).to(device)
target_seqs = torch.tensor(target_seqs, dtype=torch.long).to(device)

# Define input and output sizes based on the processed text
input_size = len(chars)
output_size = len(chars)

In [ ]:
# Initialize the model
model = CharRNN(input_size, hidden_size, output_size, num_layers).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
total_batches = len(input_seqs) // batch_size
for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    # Shuffle the data
    indices = torch.randperm(len(input_seqs))
    input_seqs = input_seqs[indices]
    target_seqs = target_seqs[indices]

    # Mini-batch training
    for batch in range(total_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        input_batch = input_seqs[start_idx:end_idx]
        target_batch = target_seqs[start_idx:end_idx]

        hidden = model.init_hidden(input_batch.size(0))
        hidden = tuple(tensor.to(device) for tensor in hidden)

        # Forward pass
        output, hidden = model(input_batch, hidden)
        loss = criterion(output, target_batch)
        running_loss += loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss every 5 epochs till 50
    if (epoch + 1) % 5 == 0:
        average_loss = running_loss / total_batches
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

Epoch [5/50], Loss: 1.0369
Epoch [10/50], Loss: 0.6060
Epoch [15/50], Loss: 0.3636
Epoch [20/50], Loss: 0.2150
Epoch [25/50], Loss: 0.1355
Epoch [30/50], Loss: 0.0995
Epoch [35/50], Loss: 0.1025
Epoch [40/50], Loss: 0.0490
Epoch [45/50], Loss: 0.0305
Epoch [50/50], Loss: 0.0141


In [ ]:
# Generate text
model.eval()
# Generate text
model.eval()
sentence_end_markers = ['.', '?']  # Add other sentence end markers if needed
end_indices = [i for i, char in enumerate(text) if char in sentence_end_markers]
start = np.random.choice(end_indices) + 1  # Add 1 to start at the next character
start_seq = text[start:start+seq_length]
generated_text = start_seq

with torch.no_grad():
    hidden = model.init_hidden(1)
    hidden = tuple(tensor.to(device) for tensor in hidden)

    input_seq = torch.tensor([char_to_index[char] for char in start_seq], dtype=torch.long).unsqueeze(0).to(device)

    num_words = len(generated_text.split())

    while num_words < 100 or not generated_text.endswith((".", "!", "?")):
        output, hidden = model(input_seq, hidden)
        probabilities = nn.functional.softmax(output, dim=1).squeeze().cpu().numpy()
        predicted_index = np.random.choice(len(chars), p=probabilities)
        predicted_char = index_to_char[predicted_index]
        generated_text += predicted_char
        input_seq = torch.tensor([[predicted_index]], dtype=torch.long).to(device)

        # Update the word count
        num_words = len(generated_text.split())

print("Generated Text:")
print(generated_text)

Generated Text:
 Recommender systems employ Deep Learning architectures to provide personalized recommendations to users, enhancing user experiences and improving direas are black boxes, understanding how they arrive at their predictions or decisions can be challenging. Researchers are working towards a crural language processing, and various domains and unleashing new possibilities for solving complex real-world problems.

Furthermore, the future of Deep Learning holds immense potential. Researchers are exploring novel approaches such as recommender systems, fraud detection, custome shing orgation and data and chatbot development. With the field of Deep Learning with other fields such as reinforcement learning, and attention mechanisms to further enhance the capabilities of Deep Learning technologies are paramount and society as a whole.


In [ ]:
start_seq

' Recommender systems employ Deep Learning architectures to provide personalized recommendations to u'